In [1]:
#!/usr/bin/env python3
"""
Import regression_data.csv that has NO header row.
- Reads CSV with header=None and assigns correct column names.
- Parses the date to YYYY-MM-DD.
- Creates database and a table without PRIMARY KEY (preserve duplicates).
- Inserts rows in chunks.

Usage:
  pip install pandas sqlalchemy pymysql
  python import_regression_headerless.py
"""

import os
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql

# ---------- CONFIG ----------
CSV_PATH = r"..\data\regression_data.csv"
DB_USER = "root"
DB_PASS = "123456"   # <<< REPLACE with your password
DB_HOST = "127.0.0.1"
DB_PORT = 3306
DB_NAME = "house_price_regression"
TARGET_TABLE = "house_price_data"   # table without PK to preserve duplicates
CHUNKSIZE = 5000
# ---------- END CONFIG ----------

# Column names in correct order for your CSV (no header in file)
COLUMN_NAMES = [
    "id","date","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view",
    "condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode","lat",
    "long","sqft_living15","sqft_lot15","price"
]

def read_csv_headerless(path):
    print("Reading CSV as headerless with names...")
    # header=None tells pandas the file has no header row; names=... assigns the column names
    df = pd.read_csv(path, header=None, names=COLUMN_NAMES, dtype=str, encoding='utf-8', low_memory=False)
    print("Raw rows read:", len(df))
    print("Columns:", df.columns.tolist())
    print("Preview:")
    print(df.head(3).to_string(index=False))
    return df

def clean_dataframe(df):
    # rename date -> sale_date for DB schema consistency
    if 'date' in df.columns:
        df = df.rename(columns={'date': 'sale_date'})

    # Trim whitespace and normalize empty strings -> None
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.replace({'': None, 'NULL': None, 'NaN': None})

    # Parse dates (handles mm/dd/yy and mm/dd/YYYY)
    if 'sale_date' in df.columns:
        df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True, errors='coerce')
        df['sale_date'] = df['sale_date'].dt.strftime('%Y-%m-%d')
        df.loc[df['sale_date'] == 'NaT', 'sale_date'] = None

    # Convert numeric columns to numeric types when reasonable
    num_cols = [
        "id","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade",
        "sqft_above","sqft_basement","yr_built","yr_renovated","lat","long","sqft_living15","sqft_lot15","price"
    ]
    for c in num_cols:
        if c in df.columns:
            if c == "id":
                # keep as integer when possible
                df[c] = pd.to_numeric(df[c], errors='coerce').astype('Int64')
            else:
                df[c] = pd.to_numeric(df[c], errors='coerce')

    # Reorder columns to stable order that matches table schema to be created
    expected = ["id","sale_date","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view",
                "condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode","lat",
                "long","sqft_living15","sqft_lot15","price"]
    available = [c for c in expected if c in df.columns]
    df = df[available]
    print("Cleaned DataFrame shape:", df.shape)
    return df

def ensure_database(engine_no_db, db_name):
    with engine_no_db.connect() as conn:
        conn.execute(text(f"CREATE DATABASE IF NOT EXISTS `{db_name}` DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))
    print(f"Database `{db_name}` ensured.")

def ensure_table(engine_db, table_name):
    # Create a table WITHOUT PRIMARY KEY in order to preserve duplicates.
    create_sql = f"""
    CREATE TABLE IF NOT EXISTS `{table_name}` (
      id BIGINT,
      sale_date DATE,
      bedrooms TINYINT,
      bathrooms DECIMAL(5,2),
      sqft_living INT,
      sqft_lot INT,
      floors DECIMAL(4,2),
      waterfront TINYINT,
      `view` TINYINT,
      `condition` TINYINT,
      grade TINYINT,
      sqft_above INT,
      sqft_basement INT,
      yr_built SMALLINT,
      yr_renovated SMALLINT,
      zipcode VARCHAR(20),
      lat DECIMAL(10,6),
      `long` DECIMAL(10,6),
      sqft_living15 INT,
      sqft_lot15 INT,
      price BIGINT,
      INDEX (zipcode),
      INDEX (sale_date),
      INDEX (price)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """
    with engine_db.connect() as conn:
        conn.execute(text(create_sql))
    print(f"Table `{table_name}` ensured (no PRIMARY KEY).")

def insert_chunks(df, engine_db, table_name, chunksize):
    total = len(df)
    inserted = 0
    print(f"Inserting {total} rows into `{table_name}` in chunks of {chunksize}...")
    for start in range(0, total, chunksize):
        chunk = df.iloc[start:start+chunksize].copy()
        # pandas will convert NA to NULL
        chunk.to_sql(table_name, con=engine_db, if_exists='append', index=False, chunksize=1000, method='multi')
        inserted += len(chunk)
        print(f"  inserted {inserted}/{total}")
    print("Insert finished.")

def main():
    if not os.path.exists(CSV_PATH):
        raise SystemExit(f"CSV file not found: {CSV_PATH}")

    # Step 1: read headerless CSV
    df = read_csv_headerless(CSV_PATH)

    # Step 2: clean and prepare
    df = clean_dataframe(df)

    # Step 3: ensure database exists
    engine_no_db = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/?charset=utf8mb4")
    ensure_database(engine_no_db, DB_NAME)

    # Step 4: engine to database
    engine_db = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4")

    # Step 5: ensure table (no PK)
    ensure_table(engine_db, TARGET_TABLE)

    # Step 6: insert
    insert_chunks(df, engine_db, TARGET_TABLE, CHUNKSIZE)

    # Final checks
    with engine_db.connect() as conn:
        total_rows = conn.execute(text(f"SELECT COUNT(*) FROM `{TARGET_TABLE}`")).scalar()
    print(f"Done. Rows in `{TARGET_TABLE}` now: {total_rows}")
    print(f"You can inspect duplicates with: SELECT id, COUNT(*) c FROM {TARGET_TABLE} GROUP BY id HAVING c>1 ORDER BY c DESC LIMIT 20;")

if __name__ == "__main__":
    main()

Reading CSV as headerless with names...
Raw rows read: 21597
Columns: ['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'price']
Preview:
        id     date bedrooms bathrooms sqft_living sqft_lot floors waterfront view condition grade sqft_above sqft_basement yr_built yr_renovated zipcode     lat     long sqft_living15 sqft_lot15  price
7129300520 10/13/14        3         1        1180     5650      1          0    0         3     7       1180             0     1955            0   98178 47.5112 -122.257          1340       5650 221900
6414100192  12/9/14        3      2.25        2570     7242      2          0    0         3     7       2170           400     1951         1991   98125  47.721 -122.319          1690       7639 538000
5631500400  2/25/15        2         1         770    10000      1

C:\Users\bruno\AppData\Local\Temp\ipykernel_10616\4270807192.py:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\bruno\AppData\Local\Temp\ipykernel_10616\4270807192.py:58: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True, errors='coerce')
C:\Users\bruno\AppData\Local\Temp\ipykernel_10616\4270807192.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True, errors='coerce')


Cleaned DataFrame shape: (21597, 21)
Database `house_price_regression` ensured.
Table `house_price_data` ensured (no PRIMARY KEY).
Inserting 21597 rows into `house_price_data` in chunks of 5000...
  inserted 5000/21597
  inserted 10000/21597
  inserted 15000/21597
  inserted 20000/21597
  inserted 21597/21597
Insert finished.
Done. Rows in `house_price_data` now: 21597
You can inspect duplicates with: SELECT id, COUNT(*) c FROM house_price_data GROUP BY id HAVING c>1 ORDER BY c DESC LIMIT 20;
